# Nama Kelompok :
1. Achmad Fadlan Bhahreisy (06.2018.1.06992)
2. Dimas Pratikto (06.2018.1.06958)
3. Muhammad Ihsan Nugroho (06.2018.1.07058)
4. M. Ilham Afriandhi (06.2018.1.06943)

In [4]:
import requests
import string
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

#Dimas
# Make a request to the website
r = requests.get('https://www.kompas.com/')
# Create an object to parse the HTML format
soup = BeautifulSoup(r.content, 'html.parser')
# Retrieve all popular news links (Fig. 1)
link = []
for i in soup.find('div', {'class':'most__wrap'}).find_all('a'):
    i['href'] = i['href'] + '?page=all'
    link.append(i['href'])
# For each link, we retrieve paragraphs from it, combine each paragraph as one string, and save it to documents (Fig. 2)
documents = []
for i in link:
    # Make a request to the link
    r = requests.get(i)
  
    # Initialize BeautifulSoup object to parse the content 
    soup = BeautifulSoup(r.content, 'html.parser')
  
    # Retrieve all paragraphs and combine it as one
    sen = []
    for i in soup.find('div', {'class':'read__content'}).find_all('p'):
        sen.append(i.text)
  
    # Add the combined paragraphs to documents
    documents.append(' '.join(sen))
    
#Ilham
import re
documents_clean = []
for d in documents:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)
    
#ihsan
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(documents_clean)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=vectorizer.get_feature_names())
df


,0,1,2,3,4,5,6,7,8,9
ac,0.000000,0.000000,0.000000,0.022848,0.000000,0.000000,0.021709,0.000000,0.000000,0.000000
ada,0.013147,0.021467,0.043280,0.043537,0.032156,0.102928,0.041366,0.016037,0.000000,0.047320
adalah,0.017428,0.028458,0.000000,0.014429,0.000000,0.027289,0.013709,0.010629,0.000000,0.000000
adanya,0.055195,0.045064,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ade,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.233704
...,...,...,...,...,...,...,...,...,...,...
yang,0.236640,0.000000,0.057707,0.119727,0.096468,0.164685,0.093073,0.136310,0.108878,0.177451
yasin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058426
ylki,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019800,0.000000,0.000000
yogyakarta,0.000000,0.000000,0.000000,0.022848,0.000000,0.000000,0.021709,0.000000,0.000000,0.000000


In [5]:
#aku
def get_similar_articles(q, df):
  print("query:", q)
  print("Berikut artikel dengan nilai cosine similarity tertinggi: ")
  # Convert the query become a vector
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  # Calculate the similarity
  for i in range(10):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  # Sort the values 
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  # Print the articles and their similarity values
  for k, v in sim_sorted:
    if v != 0.0:
      print("Nilai Similaritas:", v)
      print(documents_clean[k])
      print()
# Add The Query
q1 = 'covid'
# Call the function
get_similar_articles(q1, df)

query: covid
Berikut artikel dengan nilai cosine similarity tertinggi: 
Nilai Similaritas: 0.3293695177145925
jakarta kompas com kasus baru covid di indonesia dalam beberapa hari terakhir masih meningkat tajam skenario terburuk saat ini tengah dijalankan pemerintah untuk mengatasi penambahan pasien yang dirawat di rumah sakit dan menekan angka kematian data dari satgas penanganan covid hingga selasa pukul wib menunjukkan ada penambahan kasus baru covid dalam jam terakhir ini merupakan rekor tertinggi penambahan pasien dalam sehari selama pandemi baca juga luhut akui worst case scenario lonjakan covid telah dijalankan penambahan tersebut menyebabkan total kasus covid di indonesia saat ini mencapai orang terhitung sejak kasus pertama diumumkan presiden joko widodo pada maret informasi ini disampaikan satgas penanganan covid kepada wartawan pada selasa sore data juga bisa diakses publik melalui situs covid go id dapatkan informasi inspirasi dan insight di email kamu daftarkan email data y